In [1]:
import pandas as pd
import geopandas

In [2]:
# world_map = geopandas.read_file('filtered_countries.geojson')
# world_map.loc[world_map['name'].str.contains('France')].iloc[0,2]

In [2]:
countries = {
    'BR': {'country': 'BR', 'country_name': 'Brazil', 'latitude': -15.749997, 'longitude': -51.92528}, 
    'CA': {'country': 'CA', 'country_name': 'Canada', 'latitude': 56.130366, 'longitude': -106.346771},
    'DE': {'country': 'DE', 'country_name': 'Germany', 'latitude': 51.165691, 'longitude': 10.451526},
    'FR': {'country': 'FR', 'country_name': 'France', 'latitude': 46.227638, 'longitude': 2.213749},
    'GB': {'country': 'GB', 'country_name': 'UK', 'latitude': 55.378051, 'longitude': -3.435973},
    'IN': {'country': 'IN', 'country_name': 'India', 'latitude': 20.593684, 'longitude': 78.96288},
    'JP': {'country': 'JP', 'country_name': 'Japan', 'latitude': 36.204824, 'longitude': 138.252924},
    'KR': {'country': 'KR', 'country_name': 'S Korea', 'latitude': 35.907757, 'longitude': 127.766922},
    'MX': {'country': 'MX', 'country_name': 'Mexico', 'latitude': 23.634501, 'longitude': -102.552784},
    'RU': {'country': 'RU', 'country_name': 'Russia', 'latitude': 61.52401, 'longitude': 105.318756},
    'US': {'country': 'US', 'country_name': 'USA', 'latitude': 37.09024, 'longitude': -95.712891}}

countries_df = pd.DataFrame(countries.values())

# countries_df.to_csv('countries.csv', index = False)

# countries_df

df = pd.read_csv('df_youtube_raw.csv')

df = df.drop(columns = [df.columns[0], 'publish_time', 'thumbnail_link', 
                         'ratings_disabled', 'comments_disabled', 'description'])

df = df.merge(countries_df, on='country')
df = df.drop(columns = ['country'])

df['views'] = pd.to_numeric(df['views'], downcast='integer')
df['likes'] = pd.to_numeric(df['likes'], downcast='integer')
df['dislikes'] = pd.to_numeric(df['dislikes'], downcast='integer')
df['comment_count'] = pd.to_numeric(df['comment_count'], downcast='integer')
# df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%Y-%m-%d')

df = df.loc[df['trending_date'] > '2020-01-01']
df = df.dropna()

def transformTags(x):
#     return "|".join(x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|'))
    return x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|')

df['tags'] = df['tags'].apply(transformTags)

In [3]:
bubble_map_data = df.groupby(['country_name', 'latitude', 'longitude']).agg({'title': 'count'}).rename(columns = {'title': 'value'})

bubble_map_data.to_csv('bubble_map_data.csv', index = True)

pd.read_csv('bubble_map_data.csv')

,country_name,latitude,longitude,value
0,Brazil,-15.749997,-51.925280,46887
1,Canada,56.130366,-106.346771,46805
2,France,46.227638,2.213749,46814
3,Germany,51.165691,10.451526,46773
4,India,20.593684,78.962880,44048
5,Japan,36.204824,138.252924,46809
6,Mexico,23.634501,-102.552784,46674
7,Russia,61.524010,105.318756,45723
8,S Korea,35.907757,127.766922,43705
9,UK,55.378051,-3.435973,46846


In [4]:
doughnut = df.groupby(['country_name', 'category_title']).agg({'title': 'count'}).rename(columns = {'title': 'count'}).reset_index()

doughnut = doughnut.merge(bubble_map_data, on = 'country_name').rename(columns = {'value' : 'total'})

doughnut['value'] = round(doughnut['count'] / doughnut['total'] * 100, 1)

doughnut.index = doughnut['country_name']

doughnut = doughnut.drop(columns = ['country_name', 'count', 'total'])

doughnut_temp = df.groupby(['category_title']).agg({'title': 'count'}).rename(columns = {'title': 'value'}).reset_index()

doughnut_temp['value'] = round(doughnut_temp['value'] / doughnut_temp.sum()['value'] * 100, 1)

doughnut_temp.index = ['Overall'] * doughnut_temp.shape[0]

doughnut_temp.index.name = 'country_name'

doughnut = doughnut.append(doughnut_temp)

main_list = []

for i in doughnut.index.unique():    
    as_list = doughnut.loc[i]['category_title'].tolist()
    
    for j in range(len(as_list)):        
        if doughnut.loc[i][doughnut.loc[i].category_title.str.contains(as_list[j])]['value'][0] < 3.5:
            as_list[j] = 'Others'

    main_list += as_list


doughnut['category_title'] = main_list

doughnut = doughnut.groupby(['country_name', 'category_title']).sum().round(1).sort_values(['country_name', 'value'], ascending = [True, False])

doughnut.to_csv('doughnut.csv', index = True)

pd.read_csv('doughnut.csv')

doughnut.head(20)

value
country_name category_title             
Brazil       Music                  25.8
             Entertainment          25.5
             Sports                 15.3
             People & Blogs         12.0
             Others                  9.5
             Gaming                  7.1
             Comedy                  4.8
Canada       Entertainment          21.3
             Music                  15.8
             Sports                 12.6
             Gaming                 11.7
             Others                  9.5
             People & Blogs          8.6
             Comedy                  6.9
             Science & Technology    4.9
             Howto & Style           4.8
             News & Politics         3.9
France       Music                  24.1
             Entertainment          23.7
             Sports                 11.5

In [5]:
def flatList(x):
    return [item for sublist in x for item in sublist if len(item) > 1]

wordcloud = pd.DataFrame(df.groupby('country_name')['tags'].apply(list))
wordcloud = pd.DataFrame(wordcloud['tags'].apply(flatList))
wordcloud.loc['Overall'] = [[item for list in wordcloud['tags'].to_list() for item in list]]

def topNWords(x, n = 250):
    return pd.DataFrame(x[0]).value_counts().sort_values(ascending = False).head(n)

wordcloud = pd.DataFrame(wordcloud.groupby('country_name')['tags'].apply(topNWords)).reset_index().rename(columns = {0: 'words', 'tags': 'count'})
wordcloud.index = wordcloud.country_name
wordcloud[['words', 'count']].to_csv('wordcloud.csv', index = True)
pd.read_csv('wordcloud.csv')

# wordcloud = wordcloud[['words', 'count']].groupby('country_name')['words'].apply(list).apply(lambda x: "|".join(x))

,country_name,words,count
0,Brazil,futebol,2442
1,Brazil,vlog,1904
2,Brazil,flamengo,1597
3,Brazil,humor,1536
4,Brazil,ao vivo,1485
...,...,...,...
2995,USA,snl 46,209
2996,USA,try,209
2997,USA,sidemen,208
2998,USA,holiday,208


In [46]:
temp = df.copy()

temp['likes_dislikes'] = temp['likes'] + temp['dislikes']

bubble_plot_data = temp.groupby(['country_name']).agg({'comment_count': 'mean', 'likes': 'mean', 'dislikes': 'mean', 'views': 'mean', 'likes_dislikes': 'mean'}).rename(columns = {'comment_count': 'comments'})

bubble_plot_data.to_csv('bubble_plot_data.csv', index = True)

pd.read_csv('bubble_plot_data.csv')

,country_name,comments,likes,dislikes,views,likes_dislikes
0,Brazil,10517.950754,121177.869388,2210.934097,1.553740e+06,123388.803485
1,Canada,15027.960132,146143.476723,3282.655336,2.494640e+06,149426.132059
2,France,7956.089589,80878.094566,1570.103110,1.076321e+06,82448.197676
3,Germany,9832.231159,86969.625660,2284.854638,1.364354e+06,89254.480298
4,India,14874.329754,133201.399382,8175.394751,2.680423e+06,141376.794134
5,Japan,7414.571834,55787.095131,1346.978637,1.113999e+06,57134.073768
6,Mexico,14060.104298,149722.652333,3062.205361,2.022844e+06,152784.857694
7,Russia,5975.693852,52789.012707,1660.048203,6.608647e+05,54449.060910
8,S Korea,10399.728406,76750.184990,1513.109393,1.458075e+06,78263.294383
9,UK,11991.863959,114905.850403,2530.600137,1.931117e+06,117436.450540


In [8]:
stacked_barplot = df.groupby(['country_name', 'category_title']).agg({'comment_count': 'mean', 'likes': 'mean', 'dislikes': 'mean', 'views': 'mean'}).rename(columns = {'comment_count': 'comments'})

overall_engagement = stacked_barplot.reset_index()

overall_engagement['country_name'] = overall_engagement['country_name'].apply(lambda x: "Overall")

overall_engagement = overall_engagement.groupby(['country_name', 'category_title']).agg({'comments': 'mean', 'likes': 'mean', 'dislikes': 'mean', 'views': 'mean'}).rename(columns = {'comment_count': 'comments'})

stacked_barplot = stacked_barplot.append(overall_engagement)

stacked_barplot['engagement_rate'] = round(((stacked_barplot['comments'] / stacked_barplot['views']) + \
                                    (stacked_barplot['likes'] / stacked_barplot['views']) + \
                                    (stacked_barplot['dislikes'] / stacked_barplot['views'])) * 100, 2)

stacked_barplot['total_engagement'] = stacked_barplot['comments'] + stacked_barplot['likes'] + stacked_barplot['dislikes']

for i in list(stacked_barplot.columns[:3]):
    stacked_barplot[i] = round(stacked_barplot[i] / stacked_barplot['total_engagement'] * 100, 2)

stacked_barplot = stacked_barplot.dropna()

stacked_barplot.iloc[:,:-1].to_csv('stacked_barplot.csv', index = True)
pd.read_csv('stacked_barplot.csv')

,country_name,category_title,comments,likes,dislikes,views,engagement_rate
0,Brazil,Autos & Vehicles,4.99,94.06,0.95,2.929350e+05,11.77
1,Brazil,Comedy,3.42,95.02,1.56,9.036194e+05,10.54
2,Brazil,Education,4.81,93.59,1.60,1.754240e+05,10.08
3,Brazil,Entertainment,5.52,93.15,1.33,1.013372e+06,10.77
4,Brazil,Film & Animation,6.31,91.93,1.76,1.642020e+06,5.69
...,...,...,...,...,...,...,...
164,Overall,People & Blogs,6.92,90.43,2.65,1.176777e+06,7.13
165,Overall,Pets & Animals,7.41,91.01,1.58,6.003352e+05,4.56
166,Overall,Science & Technology,6.21,90.62,3.17,2.424623e+06,3.93
167,Overall,Sports,7.90,89.18,2.93,1.206966e+06,2.86


In [9]:
def topNChannelsViews(x, n = 100,):
    return pd.DataFrame(x.sort_values(['views'], ascending = False).head(n)['channel_title'])

def topNChannelsVideos(x, n = 100,):
    return pd.DataFrame(x.sort_values(['videos'], ascending = False).head(n)['channel_title'])

circular_barplot = df.groupby(['country_name', 'channel_title', 'video_id']).agg({'views': 'max'}).reset_index().rename(columns={'video_id': 'videos'}).dropna()

circular_barplot = circular_barplot.groupby(['country_name', 'channel_title']).agg({'videos': 'count', 'views': 'mean'}).reset_index()

circular_barplot = circular_barplot.sort_values(['videos'], ascending = False).reset_index()

top_n_channels = circular_barplot.groupby('country_name').apply(topNChannelsViews)

top_n_channels = top_n_channels.append(circular_barplot.groupby('country_name').apply(topNChannelsVideos))

top_n_channels = top_n_channels.reset_index()['channel_title']

circular_barplot = circular_barplot[circular_barplot['channel_title'].isin(top_n_channels)].sort_values(['videos'], ascending = False)


circular_barplot.iloc[:,1:5].to_csv('circular_barplot.csv', index = False)

pd.read_csv('circular_barplot.csv')

,country_name,channel_title,videos,views
0,India,Colors TV,486,1.038781e+06
1,India,Vijay Television,417,2.699302e+06
2,Russia,Телеканал Дождь,381,6.253084e+05
3,UK,Sky Sports Football,322,5.875838e+05
4,Russia,Матч ТВ,299,3.815177e+05
...,...,...,...,...
3428,Canada,Zee Music Company,1,4.913609e+07
3429,Canada,[adult swim junior],1,8.370539e+06
3430,Canada,Weki Meki 위키미키,1,9.988644e+06
3431,Canada,White Hill Music,1,9.488474e+06


In [18]:
line_chart = df.groupby(['country_name', 'channel_title', 'video_id', 'category_title', 'trending_date']).agg({'views': 'max'}).reset_index().rename(columns={'video_id': 'videos'}).dropna()

line_chart = line_chart.groupby(['country_name', 'channel_title', 'category_title', 'trending_date']).agg({'videos': 'count', 'views': 'mean'}).reset_index()

line_chart = line_chart[line_chart['channel_title'].isin(list(circular_barplot['channel_title']))]

line_chart.to_csv('line_chart.csv', index = False)

pd.read_csv('line_chart.csv')

,country_name,channel_title,category_title,trending_date,videos,views
0,Brazil,(G)I-DLE (여자)아이들 (Official YouTube Channel),Music,2020-12-28,1,410874.0
1,Brazil,(G)I-DLE (여자)아이들 (Official YouTube Channel),Music,2020-12-29,1,505597.0
2,Brazil,(G)I-DLE (여자)아이들 (Official YouTube Channel),Music,2020-12-31,1,610936.0
3,Brazil,(G)I-DLE (여자)아이들 (Official YouTube Channel),Music,2021-01-01,1,642698.0
4,Brazil,(G)I-DLE (여자)아이들 (Official YouTube Channel),Music,2021-01-02,1,669488.0
...,...,...,...,...,...,...
129987,USA,채널 NCT DANCE,Entertainment,2020-10-22,1,2837636.0
129988,USA,채널 NCT DANCE,Entertainment,2020-10-23,1,3281802.0
129989,USA,채널 NCT DANCE,Entertainment,2020-10-24,1,3703004.0
129990,USA,채널 NCT DANCE,Entertainment,2020-10-25,1,4080744.0


In [11]:
# def expand(x):
#     return x

# arr = list(df.loc[df['country_name'] == 'Germany']['tags'].apply(expand))
# all_tags = [item.lower() for a in arr for item in a]

# df_alltags = pd.DataFrame(all_tags).rename(columns = {0: 'tags'})
# all_tags_count = pd.DataFrame(df_alltags.value_counts())

In [12]:
# all_tags_count.head(40)

# all_tags_count.filter(like = 'none', axis = 0)

# df.filter(like = 'index to keep', axis=0)